# Import libraries

In [2]:
from tensorflow import keras
import tensorflow as tf
import os,datetime
import tensorflow_datasets as tfds


# Dataset loading

In [ ]:
df,info=tfds.load("rock_paper_scissors" ,with_info=True,as_supervised=True) 


# spliting the validation set from training set

In [ ]:
num_validation=0.1*info.splits['train'].num_examples
num_validation=tf.cast(num_validation, tf.int64)

# importing train and test set

In [ ]:
train_data=df['train']
test_images=df['train']

In [ ]:
train_data


# Data Preprocessing

In [ ]:
def preprocess(image,labels):
    image=tf.cast(image,tf.float32)
    image/=255.0
    return image,labels

In [ ]:
valid_train_data =train_data.map(preprocess)

In [ ]:
test_image= test_images.map(preprocess)

# Shuffling the images

In [ ]:
valid_train_data_shuffled= valid_train_data.shuffle(1000)

In [ ]:
valid_data=valid_train_data_shuffled.take(num_validation)
train_data=valid_train_data_shuffled.skip(num_validation)

In [ ]:
num_test=info.splits['test'].num_examples
numtest=tf.cast(num_test,tf.int64)

# formatting the images to our convenice

In [ ]:
batch_size= 32
train_data=train_data.batch(batch_size).prefetch(1)
valid_data=valid_data.batch(num_validation).prefetch(1)
test_images=test_images.batch(num_test).prefetch(1)

In [ ]:
train_images,train_labels=next(iter(train_data))
valid_images,valid_labels=next(iter(valid_data))
test_images,test_labels=next(iter(test_images))

In [ ]:
train_images =  tf.image.resize(train_images,(150,150))
valid_images =tf.image.resize(valid_images,(150,150))
test_images=tf.image.resize(test_images,(150,150))

In [ ]:
train_images.shape


# Data augmentation

In [ ]:
from keras.preprocessing.image import  ImageDataGenerator
datagen=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
it_gen=datagen.flow(train_images,train_labels,batch_size=32)

# model creation with Convalution layer

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten,BatchNormalization,Dropout
import numpy as np

model = Sequential([
                    Conv2D(32, 3, padding='same',  activation='relu',kernel_initializer='he_uniform', input_shape = [150, 150, 3]),
                    MaxPooling2D(2),
                    Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation='relu'),
                    MaxPooling2D(2),
                    Conv2D(64, 3, padding='same', kernel_initializer='he_uniform',activation='relu'),
                    MaxPooling2D(2),
                    Conv2D(64, 3, padding='same', kernel_initializer='he_uniform',activation='relu'),
                    MaxPooling2D(2),
                    Conv2D(128, 3, padding='same', kernel_initializer='he_uniform',activation='relu'),
                    MaxPooling2D(2),
                    Flatten(),
                    Dense(128, kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(3, activation = 'softmax'),
                    ])

In [ ]:
model.summary()

# Compling anf fitting the model

In [ ]:
steps= int(train_images.shape[0]/32)
model.compile(optimizer=optimizers.Adam(1e-3),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
early_stopping=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)
history =model.fit(it_gen, epochs=200,validation_data=(valid_images,valid_labels),callbacks=[early_stopping],steps_per_epoch=steps,verbose=2)

# Saving the model

In [ ]:
model.save("rps.h5")

In [ ]:
model_json = model.to_json()
with open("rps.json", "w") as json_file:
    json_file.write(model_json)